# CNN, CrossEntropyLoss, label encoding

### Prepare venv

In [55]:
dir = "../../dataset/variance_big_256/"
labels = ["canter", "trot", "walk"]

rng = 42

batch_size = 8
epochs = 10

#### imports

In [56]:
import numpy as np
import os
import cv2

from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import TensorDataset, DataLoader

#### load variances

In [57]:
files = sorted(os.listdir(dir))

X_list, y_list = [], []

for label in labels:
    for file in sorted(os.listdir(dir + label)):
        var = np.load(dir + label + "/" + file).astype(np.float32)
        X_list.append(var)
        y_list.append(labels.index(label))

X = np.stack(X_list, axis=0)       # shape: (N, 256, 256)
y = np.array(y_list)               # shape: (N,)

#### train/test split

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=rng)

## Tensors

### Train

In [59]:
X_tensor = torch.tensor(X_train[:, None, :, :], dtype=torch.float32)    # insert channels dimention
y_tensor = torch.tensor(y_train, dtype=torch.long)

dataset = TensorDataset(X_tensor, y_tensor)
loader = DataLoader(dataset, batch_size=8, shuffle=True)

### Test

In [60]:
X_test_tensor = torch.tensor(X_test[:, None, :, :], dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


Eye check

In [61]:
for X, y in test_loader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([8, 1, 256, 256])
Shape of y: torch.Size([8]) torch.int64


## Model

In [62]:
import torch.nn as nn

class SmallCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),  # Input: 1x256x256 → Output: 16x256x256
            nn.ReLU(),
            nn.MaxPool2d(2),                             # Output: 16x128x128

            nn.Conv2d(16, 32, kernel_size=3, padding=1), # Output: 32x128x128
            nn.ReLU(),
            nn.MaxPool2d(2),                             # Output: 32x64x64

            nn.Conv2d(32, 64, kernel_size=3, padding=1), # Output: 64x64x64
            nn.ReLU(),
            nn.MaxPool2d(2),                             # Output: 64x32x32
        )
        
        self.fc = nn.Linear(64*32*32, num_classes)
        
    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

## Training

In [63]:
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SmallCNN(num_classes=len(np.unique(y))).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)



losses = []


In [64]:

for epoch in range(epochs):
    model.train()
    running_loss = 0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for xb, yb in test_loader:
            xb, yb = xb.to(device), yb.to(device)
            preds = model(xb)
            test_loss += criterion(preds, yb).item() * xb.size(0)
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(loader):.4f}, Test loss: {test_loss/len(test_dataset):.4f}")
    losses.append([running_loss/len(loader), test_loss/len(test_dataset)])

RuntimeError: mat1 and mat2 shapes cannot be multiplied (16384x32 and 65536x3)

## Eval

In [ ]:
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

all_preds = []
all_labels = []
correct = 0
total = 0
loss_total = 0.0
criterion = torch.nn.CrossEntropyLoss()

with torch.no_grad():
    for xb, yb in test_loader:
        xb, yb = xb.to(device), yb.to(device)
        preds = model(xb)
        predicted_labels = preds.argmax(dim=1)
        
        all_preds.extend(predicted_labels.cpu().numpy())
        all_labels.extend(yb.cpu().numpy())
        
        # Compute loss
        loss = criterion(preds, yb)
        loss_total += loss.item() * xb.size(0)  # sum over batch
        
        # Compute accuracy
        predicted_labels = preds.argmax(dim=1)
        correct += (predicted_labels == yb).sum().item()
        total += yb.size(0)

avg_loss = loss_total / total
accuracy = correct / total

print(f"Test Loss: {avg_loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

# Suppose all_labels and all_preds are defined
cm = confusion_matrix(all_labels, all_preds)

# Normalize per true class
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

disp = ConfusionMatrixDisplay(confusion_matrix=cm_normalized,
                              display_labels=np.unique(all_labels))
ax = plt.gca()
disp.plot(cmap=plt.cm.Blues, ax=ax)

# Stabilize color scale
ax.images[0].set_clim(0, 1)  # force color range 0-1

plt.title("Normalized Confusion Matrix (per-class accuracy)")
plt.show()


In [ ]:
import matplotlib.pyplot as plt

skip = 1

losses_array = np.array(losses)  # shape -> (num_epochs, 2)

# Split columns
train_loss = losses_array[:, 0]
test_loss = losses_array[:, 1]

epochs = range(skip+1, len(train_loss)+1)

fig, ax = plt.subplots(figsize=(8,5))

# Plot loss on left y-axis
ax.plot(epochs, train_loss[skip:], label='Train Loss')
ax.plot(epochs, test_loss[skip:], label='Test Loss')
text = ax.set_xlabel('Epoch')
text = ax.set_ylabel('Loss')




# Save

In [ ]:
torch.save(model.state_dict(), 'model_temp.pth')